In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
%cd '/content/drive/My Drive/University Of Calgary/ENEL 645 Data Mining and Knowledge Discovery/Project/models/research'
!protoc object_detection/protos/*.proto --python_out=.
import os
os.environ['PYTHONPATH'] += ':/content/drive/My Drive/University Of Calgary/ENEL 645 Data Mining and Knowledge Discovery/Project/models/research/:/content/drive/My Drive/University Of Calgary/ENEL 645 Data Mining and Knowledge Discovery/Project/models/research/slim'
!python3 setup.py build
!python3 setup.py install

/content/drive/My Drive/University Of Calgary/ENEL 645 Data Mining and Knowledge Discovery/Project/models/research
running build
running build_py
copying object_detection/protos/anchor_generator_pb2.py -> build/lib/object_detection/protos
copying object_detection/protos/argmax_matcher_pb2.py -> build/lib/object_detection/protos
copying object_detection/protos/losses_pb2.py -> build/lib/object_detection/protos
copying object_detection/protos/train_pb2.py -> build/lib/object_detection/protos
copying object_detection/protos/bipartite_matcher_pb2.py -> build/lib/object_detection/protos
copying object_detection/protos/matcher_pb2.py -> build/lib/object_detection/protos
copying object_detection/protos/box_coder_pb2.py -> build/lib/object_detection/protos
copying object_detection/protos/mean_stddev_box_coder_pb2.py -> build/lib/object_detection/protos
copying object_detection/protos/model_pb2.py -> build/lib/object_detection/protos
copying object_detection/protos/box_predictor_pb2.py -> build

In [0]:
%cd '/content/drive/My Drive/University Of Calgary/ENEL 645 Data Mining and Knowledge Discovery/Project/models/research/object_detection'

/content/drive/My Drive/University Of Calgary/ENEL 645 Data Mining and Knowledge Discovery/Project/models/research/object_detection


In [0]:
import os
import glob

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = 'object_detector_graph/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = 'training/label_map.pbtxt'

# If you want to test the code with your images, just add images files to the PATH_TO_TEST_IMAGES_DIR (./test).
PATH_TO_TEST_IMAGES_DIR = 'data/JPEGImages-test'

assert os.path.isfile(PATH_TO_CKPT)
assert os.path.isfile(PATH_TO_LABELS)
TEST_IMAGE_PATHS = glob.glob(os.path.join(PATH_TO_TEST_IMAGES_DIR, "*.*"))
assert len(TEST_IMAGE_PATHS) > 0, 'No image found in `{}`.'.format(PATH_TO_TEST_IMAGES_DIR)
print ("Number of Test Images : ",len(TEST_IMAGE_PATHS))

Number of Test Images :  11748


In [0]:
import numpy as np
import pandas as pd
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

# This is needed since the notebook is stored in the object_detection folder.
#sys.path.append("..")
from object_detection.utils import ops as utils_ops


# This is needed to display the images.
%matplotlib inline


from object_detection.utils import label_map_util

from object_detection.utils import visualization_utils as vis_util

NUM_CLASSES = 3

detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')


label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(
    label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)


def load_image_into_numpy_array(image):
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape(
        (im_height, im_width, 3)).astype(np.uint8)


# Size, in inches, of the output images.
IMAGE_SIZE = (20, 20)


def run_inference_for_single_image(image, graph):
    with graph.as_default():
        with tf.Session() as sess:
            # Get handles to input and output tensors
            ops = tf.get_default_graph().get_operations()
            all_tensor_names = {
                output.name for op in ops for output in op.outputs}
            tensor_dict = {}
            for key in [
                'num_detections', 'detection_boxes', 'detection_scores',
                'detection_classes', 'detection_masks'
            ]:
                tensor_name = key + ':0'
                if tensor_name in all_tensor_names:
                    tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
                        tensor_name)
            if 'detection_masks' in tensor_dict:
                # The following processing is only for single image
                detection_boxes = tf.squeeze(
                    tensor_dict['detection_boxes'], [0])
                detection_masks = tf.squeeze(
                    tensor_dict['detection_masks'], [0])
                # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
                real_num_detection = tf.cast(
                    tensor_dict['num_detections'][0], tf.int32)
                detection_boxes = tf.slice(detection_boxes, [0, 0], [
                                           real_num_detection, -1])
                detection_masks = tf.slice(detection_masks, [0, 0, 0], [
                                           real_num_detection, -1, -1])
                detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
                    detection_masks, detection_boxes, image.shape[0], image.shape[1])
                detection_masks_reframed = tf.cast(
                    tf.greater(detection_masks_reframed, 0.5), tf.uint8)
                # Follow the convention by adding back the batch dimension
                tensor_dict['detection_masks'] = tf.expand_dims(
                    detection_masks_reframed, 0)
            image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

            # Run inference
            output_dict = sess.run(tensor_dict,
                                   feed_dict={image_tensor: np.expand_dims(image, 0)})

            # all outputs are float32 numpy arrays, so convert types as appropriate
            output_dict['num_detections'] = int(
                output_dict['num_detections'][0])
            output_dict['detection_classes'] = output_dict[
                'detection_classes'][0].astype(np.uint8)
            output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
            output_dict['detection_scores'] = output_dict['detection_scores'][0]
            if 'detection_masks' in output_dict:
                output_dict['detection_masks'] = output_dict['detection_masks'][0]
    return output_dict

In [0]:
def ObjectDetector (TEST_IMAGE_PATHS):
  df = pd.DataFrame(columns = ['filename','score','class', 'xmin', 'ymin', 'xmax', 'ymax'])
  indexCount = 0
  count = 0
  for image_path in TEST_IMAGE_PATHS:
    print (image_path)
    count+=1
    print (count)
    image = Image.open(image_path)
    # the array based representation of the image will be used later in order to prepare the
    # result image with boxes and labels on it.
    image_np = load_image_into_numpy_array(image)
    # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
    image_np_expanded = np.expand_dims(image_np, axis=0)
    # Actual detection.
    output_dict = run_inference_for_single_image(image_np, detection_graph)
    
    for i in range(output_dict['num_detections']):
      if (output_dict['detection_scores'][i]>0.1):
        df.loc[indexCount,'filename'] = image_path.split("/")[-1]
        df.loc[indexCount,'score'] = output_dict['detection_scores'][i]
        df.loc[indexCount,'class'] = output_dict['detection_classes'][i]
        df.loc[indexCount,'xmin'] = int(output_dict['detection_boxes'][i][0] * 800)
        df.loc[indexCount,'ymin'] = int(output_dict['detection_boxes'][i][1] * 800)
        df.loc[indexCount,'xmax'] = int(output_dict['detection_boxes'][i][2] * 800)
        df.loc[indexCount,'ymax'] = int(output_dict['detection_boxes'][i][3] * 800)
        indexCount+=1

  df['class']=df['class'].map({1:'vehicle',2:'airplane',3:'ship'})
  return df

In [0]:
TEST_IMAGE_PATHS_1 = TEST_IMAGE_PATHS[:5000]
DetectedObjects_DF_Part1 = ObjectDetector(TEST_IMAGE_PATHS_1)
DetectedObjects_DF_Part1.to_csv('detection-results-part1.csv')

In [0]:
DetectedObjects_DF_Part1.head()

,filename,score,class,xmin,ymin,xmax,ymax
0,22000.jpg,0.999484,ship,344,324,368,333
1,22000.jpg,0.999474,ship,349,270,376,278
2,22000.jpg,0.999273,ship,360,232,386,241
3,22000.jpg,0.999112,ship,293,314,319,322
4,22000.jpg,0.998557,ship,257,539,285,547


In [0]:
DetectedObjects_DF_Part1.shape

(42911, 7)

In [0]:
from google.colab import files
files.download('detection-results-part1.csv')

In [0]:
TEST_IMAGE_PATHS_2 = TEST_IMAGE_PATHS[5000:]
DetectedObjects_DF_Part2 = ObjectDetector(TEST_IMAGE_PATHS_2)
DetectedObjects_DF_Part2.to_csv('detection-results-part2.csv')

In [0]:
DetectedObjects_DF_Part2.head()

,filename,score,class,xmin,ymin,xmax,ymax
0,16521.jpg,0.580433,ship,654,232,660,246
1,16521.jpg,0.529866,ship,613,285,620,303
2,16521.jpg,0.439675,ship,611,235,618,251
3,16521.jpg,0.358013,ship,656,189,667,195
4,16521.jpg,0.353868,ship,650,149,658,159


In [0]:
DetectedObjects_DF_Part2.shape

(60227, 7)

In [0]:
from google.colab import files
files.download('detection-results-part2.csv')

In [2]:
%cd '/content/drive/My Drive/University Of Calgary/ENEL 645 Data Mining and Knowledge Discovery/Project/models/research/object_detection/data'
import pandas as pd
df_1 = pd.read_csv('detection-results-part1.csv')
df_2 = pd.read_csv('detection-results-part2.csv')
detectedObjects = pd.concat([df_1,df_2])
detectedObjects.head()

/content/drive/My Drive/University Of Calgary/ENEL 645 Data Mining and Knowledge Discovery/Project/models/research/object_detection/data


,Unnamed: 0,filename,score,class,xmin,ymin,xmax,ymax
0,0,22000.jpg,0.999484,ship,344,324,368,333
1,1,22000.jpg,0.999474,ship,349,270,376,278
2,2,22000.jpg,0.999273,ship,360,232,386,241
3,3,22000.jpg,0.999112,ship,293,314,319,322
4,4,22000.jpg,0.998557,ship,257,539,285,547
